In [ ]:
import re
import time
import pandas as pd
import numpy as np
import requests
import pickle
from bs4 import BeautifulSoup

# Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Opcional per evitar warnings de HTTPS
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


### **Extracción de empresas**

In [48]:
data = []
for year in range(2022, 2026):
    for pa in range(1, 8):
        url = f"https://startupshub.catalonia.com/investments-in-catalan-startups?pageNumber={pa}&year={year}"
        resp = requests.post(url, verify=False)
        soup = BeautifulSoup(resp.text, "html.parser")
    
        # Buscar todos los elementos con la clase 'items'
        for item in soup.find_all("div", class_="items"):
            # Extraer la URL desde el atributo `url` en la etiqueta <a>
            url_element = item.find("a", attrs={"url": True})
            url = url_element["url"] if url_element else None

            # Extraer información
            startup_name = item.find("h4").text.strip() if item.find("h4") else None
            company_name = item.find("div", class_="item-text").find_all("p")[0].text.strip()
            category = item.find("div", class_="item-text").find_all("p")[1].text.strip()
            investment_amount = item.find("div", class_="col-md-4").find("p").text.strip()
        
            investors_text = item.find("div", class_="col-md-4").find("strong")
            investors = investors_text.find_next_sibling(string=True).strip() if investors_text else None
        
            date = item.find("p", class_="date").text.strip() if item.find("p", class_="date") else None

            # Agregar datos a la lista
            data.append([startup_name, company_name, category, investment_amount, investors, date, url])

# Crear DataFrame
df = pd.DataFrame(data, columns=["Startup", "Company", "Category", "Investment", "Investors", "Date", "URL"])

In [21]:
print(df)

NameError: name 'df' is not defined

### **Extracción de info de cada empresa**

In [50]:
lista = []

In [ ]:

for i in range(0, df.shape[0]):
    url = "https://startupshub.catalonia.com/" + df.loc[i, "URL"]
    print(f"Processant [{i}]: {url}")

    try:
        # Inicialitzar driver i accedir a la pàgina
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(2)  # petita pausa per permetre càrrega

        # Petició per fer servir BeautifulSoup
        resp = requests.get(url, verify=False)
        soup = BeautifulSoup(resp.text, "html.parser")

        # Comprovació de l'adreça
        address_element = soup.find('strong', class_='company-name')
        if not address_element or not address_element.find_next_sibling(string=True):
            print(f"No hi ha adreça. Es salta.")
            continue
        address = address_element.find_next_sibling(string=True).strip()

        # Dades estàtiques via BeautifulSoup
        name = soup.find('h1', class_='big_title').text.strip() if soup.find('h1', class_='big_title') else None
        description = soup.find('meta', {'name': 'description'})['content'] if soup.find('meta', {'name': 'description'}) else None
        founded = soup.find('span', class_='founded').find('strong').text.strip() if soup.find('span', class_='founded') else None
        employees = soup.find('span', class_='employers').find('strong').text.strip() if soup.find('span', class_='employers') else None

        # Camps opcionals amb control
        def safe_text(selector, label=''):
            element = soup.find('span', class_=selector)
            return element.text.replace(label, '').strip() if element else None

        industries = safe_text('industries information-item', 'Industries:')
        technologies = safe_text('technologies information-item', 'Technologies:')
        others_fields = safe_text('otherFields information-item', 'Other fields:')
        financial_founded = safe_text('founded information-item', 'Founded:')
        business_model = safe_text('model information-item', 'Business model:')
        targets = safe_text('target information-item', 'Target:')
        spinoff = safe_text('spinoffs information-item', 'Spinoff participants:')

        # Rondas de finançament
        funding_info = []
        for funding in soup.find_all('div', class_='item punts-servei-content row-fluid'):
            amount = funding.find('h3').text.strip()
            date = amount.split('(')[-1].replace(')', '')
            funding_info.append({'amount': amount, 'date': date})

        # Elements dinàmics amb Selenium + WebDriverWait
        try:
            financial_funding_stage = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "fundingStageTxt"))
            ).text
        except TimeoutException:
            print("No s'ha trobat fundingStageTxt")
            financial_funding_stage = None

        try:
            financial_employees = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "employeesTxt"))
            ).text
        except TimeoutException:
            print("No s'ha trobat employeesTxt")
            financial_employees = None

        try:
            founding = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "generic-container"))
            )[0].text
        except TimeoutException:
            print("No s'ha trobat generic-container")
            founding = None

        # Guardar dades
        data = {
            'Name': [name],
            'Description': [description],
            'Address': [address],
            'Industries': [industries],
            'Technologies': [technologies],
            'Other fields': [others_fields],
            'Funding stage': [financial_funding_stage],
            'Founded': [financial_founded],
            'Employees': [financial_employees],
            'Business model': [business_model],
            'Target': [targets],
            'Spinoff participants': [spinoff],
            'Funding': [funding_info],
            'Funding2': [founding]
        }

        lista.append(data)

    except Exception as e:
        print(f"Error inesperat a [{i}]: {e}")

    finally:
        driver.quit()


In [59]:
import pickle

with open('lista.pkl', 'wb') as f:
    pickle.dump(lista, f)

In [ ]:
df_info = []
with open('lista.pkl', 'rb') as f:
    lista = pickle.load(f)
df_info = pd.DataFrame(lista)

In [ ]:
print(df_info)

In [63]:
df_info = pd.DataFrame(lista)
df_info["Name"] = [d[0] for d in df_info["Name"]]
df_info["Description"] = [d[0] for d in df_info["Description"]]
df_info["Address"] = [d[0] for d in df_info["Address"]]
df_info["Industries"] = [d[0] for d in df_info["Industries"]]
df_info["Technologies"] = [d[0] for d in df_info["Technologies"]]
df_info["Other fields"] = [d[0] for d in df_info["Other fields"]]
df_info["Funding stage"] = [d[0] for d in df_info["Funding stage"]]
df_info["Founded"] = [d[0] for d in df_info["Founded"]]
df_info["Employees"] = [d[0] for d in df_info["Employees"]]
df_info["Business model"] = [d[0] for d in df_info["Business model"]]
df_info["Target"] = [d[0] for d in df_info["Target"]]
df_info["Spinoff participants"] = [d[0] for d in df_info["Spinoff participants"]]

In [72]:
datosFinal = df.merge(df_info, left_on='Company', right_on='Name')
datosFinal.to_pickle("datosFinal.pkl")

In [7]:
datosFinal = pd.read_pickle("datosFinal.pkl")
print(datosFinal.shape)

(466, 21)


In [18]:
#print(datosFinal.head(3))
print(datosFinal[datosFinal["Startup"].str.strip().str.upper() == "TRAVELPERK"])

        Startup        Company              Category Investment  \
2    TRAVELPERK  TRAVELPERK SL  Traveltech & Leisure   241.9M €   
289  TRAVELPERK  TRAVELPERK SL  Traveltech & Leisure      95M €   
420  TRAVELPERK  TRAVELPERK SL  Traveltech & Leisure     190M €   

                                                                            Investors  \
2    Kinnevik, General Catalyst Partners, business angels (Gilian Tans, Joel Cutler).   
289                                                 Softbank, Kinnevik, Felix Capital   
420                                                     Atomico, EQT Growth, Kinnevik   

             Date                                URL           Name  \
2    January 2022  startup/barcelona/travelperk/1195  TRAVELPERK SL   
289  January 2024  startup/barcelona/travelperk/1195  TRAVELPERK SL   
420  January 2025  startup/barcelona/travelperk/1195  TRAVELPERK SL   

                                                                                         

In [13]:
print(datosFinal.loc[2, "Funding2"][0].split("\n"))

['Funding', '190M € (JANUARY 2025)', 'Capital', 'Investors: Atomico, EQT Growth, Kinnevik', '95M € (JANUARY 2024)', 'Capital', 'Investors: Softbank, Kinnevik, Felix Capital', '241.9M € (JANUARY 2022)', 'Minority stake unknown %', 'Investors: Kinnevik, General Catalyst Partners, business angels (Gilian Tans, Joel Cutler).', '132M € (APRIL 2021)', 'Minority stake unknown %', 'Investors: Greyhound Capital', '53M € (JULY 2019)', 'Minority stake unknown %', 'Investors: KINNEVIK, PARTNERS OF DST GLOBAL, TARGET GLOBAL, FELIX CAPITAL, SUNSTONE, LOCALGLOBE', '38M € (OCTOBER 2018)', 'Minority stake unknown %', 'Investors: KINNEVIK, YURI MILNER, TOM STAFFORD, FELIX CAPITAL, TARGET GLOBAL, SPARK CAPITAL, LOCALGLOBE, SUNSTONE, AMPLO', '17M € (APRIL 2018)', 'Minority stake unknown %', 'Investors: TARGET GLOBAL, FELIX CAPITAL, SPARK CAPITAL, SUNSTONE', '6.1M € (JUNE 2016)', 'Minority stake unknown %', 'Investors: SPARK CAPITAL PARTNERS LLC, SUNSTONE CAPITAL A/S, LOCALGLOBE LLP', 'Source: Press releas

In [ ]:
conteoRegistros = []
capitales = []
investors = []

for j in range(datosFinal.shape[0]):
    lista = datosFinal.loc[j, "Funding2"][0].split("\n")
    capital_prev_values = [lista[i - 1] for i, val in enumerate(lista) if val.lower() == 'capital' and i > 0]
    investors_cleaned = [val.replace('Investors:', '').strip() for val in lista if 'Investors' in val]
    conteoRegistros.append(len(capital_prev_values))
    capitales.append(capital_prev_values)
    investors.append(investors_cleaned)

capitales = [item for sublista in capitales for item in sublista]
investors = [item for sublista in investors for item in sublista]

print(capitales)
print(investors)

In [ ]:
df_repetido = datosFinal.loc[datosFinal.index.repeat(conteoRegistros)].reset_index(drop=True)
df_repetido["capital_prev"] = capitales
df_repetido["investors"] = investors

In [ ]:
df_repetido = datosFinal.loc[datosFinal.index.repeat(conteoRegistros)].reset_index(drop=True)
print(df_repetido.iloc[0:0])
print("df_repetido:", len(df_repetido))
print("capitales:", len(capitales))
print("investors:", len(investors))


In [94]:
datosFinal = df_repetido.drop(columns=['Funding', 'Funding2'])

In [ ]:
datosFinal.to_csv('datosFinal.csv', index=False)
datosFinal.to_pickle("./datosFinal.pkl") 